In [7]:
from tensorflow.keras.models import load_model
import cv2
import mediapipe as mp
import numpy as np

emotion_model = load_model("rafdb_emotion_model.h5")  

# Emotion categories (RAF-DB)
emotions = ['Fear','Neutral', 'Angry','Disgust' , 'Happy', 'Sad', 'Surprise']

# Initialize MediaPipe for holistic detection
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

# Function for emotion recognition
def recognize_emotion(face):
    # Preprocess face for the RAF-DB trained model
    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    face = cv2.resize(face, (48, 48))             # Resize to 48x48
    face = face / 255.0                           # Normalize to [0, 1]
    face = np.expand_dims(face, axis=0)           # Add batch dimension
    face = np.expand_dims(face, axis=-1)          # Add channel dimension
    predictions = emotion_model.predict(face)
    return emotions[np.argmax(predictions)]       # Return predicted emotion

def start_detection():
    cap = cv2.VideoCapture(0)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Convert the BGR image to RGB for MediaPipe processing
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = holistic.process(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.face_landmarks:
                # Get face bounding box
                face_landmarks = results.face_landmarks.landmark
                ih, iw, _ = frame.shape
                face_coords = [(int(lm.x * iw), int(lm.y * ih)) for lm in face_landmarks]
                x_min = min([x for x, y in face_coords])
                x_max = max([x for x, y in face_coords])
                y_min = min([y for x, y in face_coords])
                y_max = max([y for x, y in face_coords])

                # Extract and preprocess the face
                face = frame[y_min:y_max, x_min:x_max]
                if face.shape[0] > 0 and face.shape[1] > 0:
                    # Predict emotion
                    emotion = recognize_emotion(face)
                    # Display emotion label on the frame
                    cv2.putText(image, emotion, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Show the video feed with emotion recognition
            cv2.imshow("Emotion Recognition and Movement Classification", image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    start_detection()


1/1 [==============================] - 0s 18ms/step
